# Beamforming - Contre-mesures RADAR
### Maxime BARRET


Avec l'aide de :
#### 'Signaux et Sytèmes' - Prof. Dr Ir Xavier Neyt 
#### 'Source Localization with sensor arrays' - Assistant Professors Charles Vanwynsberghe & Angélique Drémeau
#### Polarized Uniform Linear Array System: Beam Radiation Pattern, Beamforming Diversity Order, and Channel Capacity, Xin Su and KyungHi Chang

In [1]:
%matplotlib notebook
#Indique qu'on va afficher des figures, et permet de les modifier

import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rd     
from scipy.signal import find_peaks
import time as t
import warnings
 #Imports d'usage

### Situation initiale

La situation est la suivante : 
- Nous émettons à l'aide Uniform Linear Array (ULA) décrit ci-après
- Nous avons une source, émetteur.
- Cette source doit éclairer deux cibles, dont les échos sont correlés. 
- 10 antennes dans mon array source
- d : distance entre mes antennes dans mon array
- Le problème est plan



### Description de notre ULA

Le principe du Beamforming est le même que dans le DoANotebook, ici on va juste jouer sur la phase de chaque émetteur pour créer le faisceau. 

![](Beam.jpg)



Le principe utilisé pour générer un faisceau est simple : chaque émetteur de notre array va générer un même signal déphasé de tel sorte que deux émetteurs consécutifs aient une ddc de  $ dsin(\theta) $

### Déclaration de nos variables

En choisissant nos variables, nous devons faire attention à plusieurs choses :
- Rester cohérent avec la réalité : On ne va pas mettre 100000000 de capteurs, ni avoir une fréquence de 2 Hz
- Vérifier la condition d'échantillonnage spatial, ici : $ d < \dfrac{\lambda}{2} ) $


In [8]:
t1 = t.time()
c0 = 3e8   #célérité de la lumière dans l'air 
d = 0.4          #Distance entre mes émetteurs
M1 = 10            #Nombre d'émetteurs sur un RADAR ennemi
N = 4*721
angle = np.linspace(-np.pi/2,np.pi/2,N)     #Grille sur laquel on va chercher l'angle d'incidence

theta0 = 0   #Angle d'incidence auquel se situe le radar qu'on doit tromper : là, il est en face de la cible.
lam = 1  #Correspond à une fréquence de 300 Mhz, sachant qu'un radar se situe entre 300Mhz et 15 Ghz
         #Attention, la résolution du beamforming baisse si la fréquence est faible

#A priori, ce sont des paramètres qui ne sont pas amenés à bouger.

In [9]:
from BeamformingMethods import a, compute_A, DP


## I-  Direction d'émission

Dans un premier temps, nous allons illustrer la direction choisie.
Mes émetteurs, au nombre de 10, vont émettre un même signal s, mais déphasé, afin de générer un faisceau.
C'est un classic beamformer.

$$ \mathbf{s}(t) = \mathbf{A}_s * cos(\dfrac{2 \pi c_0}{\lambda} t)   + \mathbf{b} $$ 


Le déphasage entre 2 émetteurs consécutifs sera de 
$$ \mathbf{\Delta\phi} = \dfrac{2\pi d}{\lambda}sin(\theta) $$
pour traduire la différence de chemin de $ dsin(\theta) $

In [28]:
theta_e =  -np.pi/4  
sig_n = 3e-1   #Ecart-type de notre bruit, que je vais considérer Gaussien et additif

temp = rd.randn(M1)
s = [np.cos(2*np.pi*c0*i/lam) for i in range(M1)]
n = sig_n*(rd.randn(M1))  #Gaussien centré
delta_phi = 2*np.pi*d*np.sin(theta_e)/lam
steer = a(theta_e,lam,M1)
steer2 = compute_A([theta_e],lam,M1)
S = steer@s +n
